# Automated pipeline to run library fringe corrections in stage 2
Takes a BFE-corrected _rate_ file and can apply only the 'grid' correction (directly applying ramp solutions from the closest pointing). Performance currently depends on proximity to the 10-Lac grid. This is less of an issue in longer wavelengths, but the lower S/N of the reference files in channel 4 means performance degrades there too.

In [ ]:
crds_cache_path = '' ## Set path to cache of JWST pipeline

### Imports

In [ ]:
from astropy.io import fits
import numpy as np
from core.distortion import d2cMapping
from os import listdir
from core.funcs import point_source_centroiding

from jwst.pipeline import Spec2Pipeline

import core.linearity_coeff_ref_files as lin_ref
import core.flux_cal as phot

import os
os.environ['CRDS_CONTEXT'] = '' #Set pmap
os.environ['CRDS_PATH'] = crds_cache_path #set cache path (defined in cell above)
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu/'

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib notebook

### Set directories and files to run

In [ ]:
##Change

dataDir = [] #list of input directories (one per file)

scifile = [] #list of files to reduce (should be rate)

version = '01.05.00' #version of fringe to use, 01.05.00 is currently the only valid option

# Give list of coordinates if known (omits centroiding, useful for fainter targets)
# Format for known coordinates is list in same order as list of files, e.g.:
# alpha_list = [0.1,0.4 ...]
alpha_list = None #Put None if not known, otherwise alpha (along-slice) coordinate in arcsec
beta_list = None #Put None if not known, otherwise beta (across-slice) coordinate in arcsec

refdir = './references/FRINGE/'
photomdir = './references/PHOTOM/'
outdir = '' #output directory (relative to input directory)
distVers = 'flt8'

### Do not touch

In [ ]:
d2cMaps = {}

for b in ['1A','1B','1C','2A','2B','2C','3A','3B','3C']:
    d2cMaps[b] = d2cMapping(b,'./references/DISTORTION/',slice_transmission='10pc',fileversion = distVers)

In [ ]:
fringe_file = {}

alpha = {}
beta = {}

for i in range(len(scifile)):
    
    alpha[i] = {}
    beta[i] = {}
    fringe_file[i] = {}
    
    hdu = fits.open(dataDir[i] + scifile[i])

    detector = hdu[0].header['DETECTOR']
    subbandl = hdu[0].header['BAND']
    dithdir = hdu[0].header['DITHDIRC']
    n = (scifile[i].split('_')[2])[-1]

    if subbandl == 'SHORT':
        subband = 'A'
    elif subbandl == 'MEDIUM':
        subband = 'B'
    else:
        subband = 'C'

    if detector == 'MIRIFUSHORT':
        band = ['1{}'.format(subband),'2{}'.format(subband)]
    else:
        band = ['3{}'.format(subband),'4{}'.format(subband)]
        
    data = hdu['SCI'].data

    hdu.close()
    
    for b in band:
        print(b,n)
        
        if b[0] in ['1','2','3']:
            
            if alpha_list is None:
                try:
                    alpha[i][b],beta[i][b] = lin_ref.return_centroids(data,b,d2cMaps[b],'hi')
                except:
                    alpha[i][b],beta[i][b] = None,None
                    print('Centroiding failed, using central grid point...')
            else:
                print('Using predefined centroid...')
                
                alpha[i][b] = alpha_list[i]
                beta[i][b] = beta_list[i]
                
            fringe_file[i][b] = lin_ref.find_nearest_grid_fringe(dataDir[i]+scifile[i],alpha[i][b],beta[i][b],b,refdir,version)

    

In [ ]:
## Do fringe correction

for i in range(len(scifile)):
    print(scifile[i])

    hdu = fits.open(dataDir[i] + scifile[i])

    detector = hdu[0].header['DETECTOR']
    subbandl = hdu[0].header['BAND']
    dithdir = hdu[0].header['DITHDIRC']
    n = (scifile[i].split('_')[2])[-1]

    if subbandl == 'SHORT':
        subband = 'A'
    elif subbandl == 'MEDIUM':
        subband = 'B'
    else:
        subband = 'C'

    if detector == 'MIRIFUSHORT':
        band = ['1{}'.format(subband),'2{}'.format(subband)]
    else:
        band = ['3{}'.format(subband),'4{}'.format(subband)]

    data = hdu['SCI'].data
    err = hdu['ERR'].data

    hdu.close()

    if detector == 'MIRIFUSHORT':
        for b in band:
            fringe_hdu = fits.open(refdir + fringe_file[i][b])
            fringe_flat = fringe_hdu[1].data
            err_fringe = fringe_hdu[2].data
            fringe_hdu.close()

            if b[0]=='1':
                err[:,:512] = np.abs(data[:,:512]/fringe_flat[:,:512]) * np.sqrt((err[:,:512]/data[:,:512])**2 + (err_fringe[:,:512]/fringe_flat[:,:512])**2)
                data[:,:512] = data[:,:512]/fringe_flat[:,:512]
            else:
                err[:,512:] = np.abs(data[:,512:]/fringe_flat[:,512:]) * np.sqrt((err[:,512:]/data[:,512:])**2 + (err_fringe[:,512:]/fringe_flat[:,512:])**2)
                data[:,512:] = data[:,512:]/fringe_flat[:,512:]
    else:
        fringe_flat = fits.getdata(refdir + fringe_file[band[0],n][0])
        data = data/fringe_flat

    hdu = fits.open(dataDir[i] + scifile[i])
    hdu['SCI'].data = data
    hdu['ERR'].data = err

    hdu.writeto(dataDir[i] + outdir + scifile[i],overwrite=True)

    hdu.close()

## Take input and run spec2

In [ ]:
def run_pipeline(list_of_files, input_dir, output_dir, step=2):

    """ Apply the spectrophotometric calibration to the data
    """

    for i in range(len(list_of_files)):
        file=list_of_files[i]
        
        if step==2:

            dist_dir = './references/DISTORTION/'

            hdu = fits.open(input_dir[i]+file)
            ifu = hdu[0].header['DETECTOR']
            subbandl = hdu[0].header['BAND']
            dithdirc = hdu[0].header['DITHDIRC']

            pipe = Spec2Pipeline()
            pipe.bkg_subtract.skip = True
            pipe.master_background_mos.skip = True

            pipe.fringe.skip = True
            pipe.residual_fringe.skip = True

            #generate pixel flat and find best photom file
            dithnum = (file.split('_')[2])[-1]
            if ifu == 'MIRIFULONG':
                channel = 'long'
            elif ifu == 'MIRIFUSHORT':
                channel = 'short'
                b = '1'
                
            if subbandl == 'SHORT':
                subband = 'A'
            elif subbandl == 'MEDIUM':
                subband = 'B'
            else:
                subband = 'C'
            
            phot.gen_custom_pixel_flat(input_dir[i]+file,dist_dir,crds_cache_path,alpha[i],beta[i],dithnum,dist_ver=distVers,version=version)
            pipe.flat_field.user_supplied_flat = './references/PIXEL/custom_pixel_ref_{}_{}.fits'.format(channel,dithnum)
            pipe.flat_field.skip = False
            pipe.straylight.skip = False
            
            if ifu == 'MIRIFULONG':
                pipe.photom.override_photom = photomdir+'{}_{}_PS_PHOTOM_{}_{}.fits'.format(ifu,subband,dithdirc,distVers,version)
            else:
                pipe.photom.override_photom = photomdir+ phot.gen_photom(input_dir[i]+file,dist_dir,alpha[i][b+subband],beta[i][b+subband],dithnum,dist_ver=distVers,version=version)
            
            pipe.photom.skip = False
            pipe.photom.mrs_time_correction = True
            pipe.cube_build.skip = True
            pipe.extract_1d.skip = True
            try:
                pipe.badpix_selfcal.skip = True
                pipe.nsclean.skip = True
            except:
                pass
            pipe.pixel_replace.skip = True

            pipe.output_dir = input_dir[i] + output_dir
            pipe.save_results = True
            result = pipe.run(input_dir[i] + output_dir + file)


In [ ]:
run_pipeline(scifile, dataDir, outdir, step=2)